In [1]:
! pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment-1: Train Logistic Regression Classifier

In [5]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment-2: Train Random Forest Classifier

In [6]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       270
           1       0.91      0.67      0.77        30

    accuracy                           0.96       300
   macro avg       0.94      0.83      0.87       300
weighted avg       0.96      0.96      0.96       300



### Experiment-3: Train XGBoost Classifier

In [7]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [8]:
! pip install imblearn

Defaulting to user installation because normal site-packages is not writeable


In [9]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [10]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



---

### Track Experiment Using MLflow

In [11]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [12]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [13]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [15]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection Experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/08/19 10:23:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/19 10:23:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/558581700452758931/runs/49e4563534924fe8b90f847a24492e95.
2025/08/19 10:23:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/558581700452758931.
2025/08/19 10:24:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/19 10:24:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/558581700452758931/runs/06e0d5767a2e4f6191a7c06efbfd04e6.
2025/08/19 10:24:08 INFO mlflow.tracking._tracking_service.client: 🧪 View exp

### Register the Model

In [16]:
model_name="XGB-Smote"
run_id=input("Enter the Run ID: ")
model_uri=f"runs:/{run_id}/{model_name}"

result=mlflow.register_model(
    model_uri, model_name
)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/08/19 10:26:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 3
Created version '3' of model 'XGB-Smote'.


### Load the Model

In [19]:
import xgboost as xgb

In [21]:
model_uri = "runs:/405955c868dc4d1d82d7376b857207b7/model"
loaded_model = mlflow.xgboost.load_model(model_uri)

y_pred = loaded_model.predict(X_test)

print(y_pred[:5])

[0 0 0 0 0]


### Dev Environment --> Production

In [23]:
import mlflow
from mlflow.tracking import MlflowClient

# Define source and destination
dev_model_uri = f"models:/{model_name}@challenger"
prod_model = "anomaly-detection-prod"

# Create MLflow client
client = MlflowClient()

# Copy model version
client.copy_model_version(
    src_model_uri=dev_model_uri,
    dst_name=prod_model
)

Successfully registered model 'anomaly-detection-prod'.
Copied version '3' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1755580657195, current_stage='None', description='', last_updated_timestamp=1755580657195, name='anomaly-detection-prod', run_id='405955c868dc4d1d82d7376b857207b7', run_link='', source='models:/XGB-Smote/3', status='READY', status_message='', tags={}, user_id='', version='1'>

In [26]:
model_uri = "runs:/405955c868dc4d1d82d7376b857207b7/model"
# Load model
champion_model = mlflow.xgboost.load_model(model_uri)

y_pred = champion_model.predict(X_test)
print(y_pred[:5])

[0 0 0 0 0]
